In [19]:

# Start by creating a new Notebook for this assignment.
# setup import
%matplotlib inline

import pandas as pd 

import requests
import io

import matplotlib.pyplot as plt

import pandas as pd
import requests
import io
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
table=pd.DataFrame(tables[0])
table=table.drop(table[table.Borough=="Not assigned"].index)
table = table.groupby('Postcode').agg({'Borough':'first','Neighbourhood': ', '.join}).reset_index()
table['Neighbourhood'] = table.apply(lambda x: x['Borough'] if x['Neighbourhood']=='Not assigned' else x['Neighbourhood'], axis=1)



#in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

# rename the first column to allow merging dataframes on Postcode
c.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(c, table, on='Postcode')

# reorder column names and show the dataframe
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]

#clustering as per word toronto in borough coulmn
toronto_data=df[(df["Borough"]=="Downtown Toronto")|(df["Borough"]=="East Toronto")|(df["Borough"]=="West Toronto")|(df["Borough"]=="Central Toronto")].reset_index(drop=True)
toronto_data.head()

#getting lat,long
address="Toronto,NY"
geolocator=Nominatim(user_agent="ny_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

#generate map accordingly
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=11)
for lat,lng,label in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Neighbourhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color="blue",fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_toronto)

map_toronto